In [1]:
import sys
import pandas as pd
import re
import warnings
warnings.filterwarnings("ignore")
from sklearn import preprocessing
import gensim
from gensim.models.doc2vec import Doc2Vec, LabeledSentence
TaggededDocument = gensim.models.doc2vec.TaggedDocument

In [8]:
class cathay_embedding_lang_learners(object):
    def __init__(self, embedding_size):
        self.size=embedding_size
        self.x_train = []
        self.words2vec =[]    
        self.seq_dict = {0:'a',1:'b',2:'c',3:'d',4:'e',5:'f',6:'g',7:'h',8:'i',9:'j',10:'k',11:'l',
                    12:'m',13:'n',14:'o',15:'p',16:'q',17:'r',18:'s',19:'t',20:'u',21:'v',22:'w',23:'x',24:'y',25:'z'}
        self.alpha_seq = []
        self.slot_num = 26
        self.arr = ''
        
    def norm_numerical_ts(self, data):
        min_max_scaler = preprocessing.MinMaxScaler(feature_range=(0.01, 0.99))

        for i in data.columns:
            if data[i].values[0] != '@':
                x = data[i].values.reshape(-1, 1) 
                x_scaled = min_max_scaler.fit_transform(x)
                data[i] = x_scaled
        self.ts_data = data

    def alphabetical_seq(self):     
        
        # Put array elements in different alphabet 
        for row in range(len(self.ts_data)):
            for col in range(len(self.ts_data.columns)):
                if self.ts_data.iloc[row,col] == '@':
                    self.arr = self.arr + '@'
                else:
                    self.arr = self.arr + self.seq_dict[int(self.slot_num*self.ts_data.iloc[row,col])]
            self.alpha_seq.append(self.arr)   
            self.arr = ''

    def remove_mark(self):
        for i in range(len(self.alpha_seq)):
            self.alpha_seq[i] = self.alpha_seq[i].replace('@',' ')

    def words_to_vec(self):

        for i,sub_list in enumerate(self.alpha_seq):
            document = TaggededDocument(sub_list, tags=[i])
            self.x_train.append(document)

        model_dm = Doc2Vec(self.x_train, min_count=1, dm=1, dm_mean=1, window = 4, vector_size = self.size, sample=1e-3, negative=5, workers=4,hs=1,epochs=6)
        model_dm.train(self.x_train, total_examples=model_dm.corpus_count, epochs=70)

        for i in range(len(self.x_train)):
            self.words2vec.append(model_dm.docvecs[i])
        self.words2vec

In [9]:
if __name__ == '__main__':
    ts_data = pd.read_csv('data/car_insurance_data1.csv', sep = ',', header=None)
    #ts_data = pd.read_csv('data/'+sys.argv[1]+'.csv', sep = ',', header=None) # for .py
    len_of_ref_seq = len(ts_data)
    for i in range(1,145,4):
        ts_data[i] = '@'

In [ ]:
    embedding_size = 10
    CELL = cathay_embedding_lang_learners(embedding_size)
    CELL.norm_numerical_ts(ts_data) ### step 1, normalize a numerical series to a given range
    CELL.alphabetical_seq()         ### step 2, transform the scaled numerical series to an alphabetical sequence
    CELL.remove_mark()              ### step 3, split the letters to form a word
    CELL.words_to_vec()             ### step 4, transform a word to a vector
    result = pd.DataFrame(CELL.words2vec)
    

In [7]:
    result.to_csv("data/features_embedding_car_insurance_data1.csv", index = None)
    #result.to_csv("data/features_embedding_"+sys.argv[1]+".csv", index = None) # for .py
    